In [1]:
from SALib.sample import latin, saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import pandas as pd
import numpy as np

Setting up the problem definition for the Ishigami function.

Here we run an example to show SALib is working as expected.

In [2]:
num_samples = 1000
seed_value = 101

problem_wo_dummy = {
  'num_vars': 3,
  'names': ['x1', 'x2', 'x3'],
  'bounds': [[-np.pi, np.pi]]*3
}

In [3]:
# Generate samples
param_values_wo_dummy = latin.sample(problem_wo_dummy, num_samples, seed=seed_value)

# Run model (example)
Y = Ishigami.evaluate(param_values_wo_dummy)

Si = sobol.analyze(problem_wo_dummy, Y, print_to_console=False)

In [4]:
total, first, second = Si.to_df()

In [5]:
total

,ST,ST_conf
x1,0.976732,0.240514
x2,1.006254,0.350020
x3,1.006333,0.291857


In [6]:
first

,S1,S1_conf
x1,0.124795,0.265309
x2,0.069378,0.203661
x3,0.112679,0.225504


Now we create a wrapping interface that accepts the dummy parameter input (the 4th parameter `x4`), but strips the parameter values from the input array. This has the effect of making the fourth parameter inactive, and therefore insensitive

In [7]:
def ishigami_wrapper(param_values):
    strip_dummy_param = param_values[:, 0:3]
    return Ishigami.evaluate(strip_dummy_param)

SALib problem definition for the above

In [8]:
problem_with_dummy = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-100.0, 100.0]]*4
}

You will see below that parameter `x4` comes out as being sensitive even though it should have zero effect

In [9]:
# Generate samples
dummy_param_values = latin.sample(problem_with_dummy, num_samples, seed=seed_value)

# Run model (example)
Y = Ishigami.evaluate(dummy_param_values)

Si = sobol.analyze(problem_with_dummy, Y, print_to_console=False)

total, first, second = Si.to_df()

In [10]:
# Results for 1,000 samples
total

,ST,ST_conf
x1,0.815695,0.361809
x2,0.879699,0.356722
x3,1.085482,0.345790
x4,1.026862,0.405504


In [11]:
first

,S1,S1_conf
x1,0.065803,0.224840
x2,-0.068657,0.238723
x3,-0.030692,0.296359
x4,-0.039045,0.285613


In [12]:
second

,S2,S2_conf
"(x1, x2)",-0.136147,0.367573
"(x1, x3)",-0.046409,0.284646
"(x1, x4)",-0.117692,0.324365
"(x2, x3)",-0.041796,0.380164
"(x2, x4)",0.093704,0.348352
"(x3, x4)",-0.388230,0.460276


Saltelli sampling does not appear to be affected, but the computational cost to run this analysis - for the purpose of detecting completely insensitive parameters - is comparatively high; almost double. 

This can be expected to exponentially increase with the number of parameters

**With dummy parameters**

In [13]:
%%time
# Run model via the wrapper
Y = ishigami_wrapper(dummy_param_values)

Wall time: 3.02 ms


In [14]:
%%time
Si = sobol.analyze(problem_with_dummy, Y, print_to_console=False)

Wall time: 14 ms


In [15]:
total, first, second = Si.to_df()

total

,ST,ST_conf
x1,0.815695,0.357648
x2,0.879699,0.361047
x3,1.085482,0.432636
x4,1.026862,0.423163


In [16]:
first

,S1,S1_conf
x1,0.065803,0.245513
x2,-0.068657,0.259097
x3,-0.030692,0.304848
x4,-0.039045,0.303142


**Without dummy parameters**

In [17]:
%%time
# Run model directly
Y = Ishigami.evaluate(param_values_wo_dummy)

Wall time: 2 ms


In [18]:
%%time
Si = sobol.analyze(problem_wo_dummy, Y, print_to_console=False)

Wall time: 8 ms
